# Analyse der log_files

In [82]:
# import important modules
import os
import csv
import pandas as pd

In [83]:
test=pd.read_csv('/home/jan/Documents/Doktorarbeit/Daten/logfiles/HOAF_01-IPT_Kinect_2018_06_26_p2.log', delimiter='\t', skiprows=3) #define test: read log in csv without tabs, skip the first three rows

In [84]:
test

,Subject,Trial,Event Type,Code,type(str),question_number(num),block_name(str),sequence(num),solution(str),answer_options(str),...,duration_question(num),Time,TTime,Uncertainty,Duration,Uncertainty.1,ReqTime,ReqDur,Stim Type,Pair Index
0,HOAF_01,0,Manual,Wartepause_2,Wartepause_2,NaN,NaN,NaN,NaN,NaN,...,NaN,26881.0,26881.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOAF_01,1,Picture,between_no_vid_trial,between_no_vid_trial,NaN,NaN,NaN,NaN,NaN,...,NaN,26996.0,0.0,1.0,1000.0,1.0,0.0,next,miss,0.0
2,HOAF_01,1,Response,10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,82408.0,55413.0,67.0,NaN,NaN,NaN,NaN,NaN,NaN
3,HOAF_01,1,Manual,Verbal_Feedback,Verbal_Feedback,NaN,NaN,NaN,NaN,NaN,...,NaN,82650.0,55655.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOAF_01,108,Response,10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100763.0,118.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,HOAF_01,110,Manual,Wait for BUTTON__3,Wait for BUTTON__3,NaN,NaN,NaN,NaN,NaN,...,NaN,100980.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,HOAF_01,110,Response,10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,109322.0,8345.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,HOAF_01,111,Manual,Wait for PULSE_1,Wait for PULSE_1,NaN,NaN,NaN,NaN,NaN,...,NaN,109488.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,HOAF_01,111,Pulse,23,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,375511.0,266035.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
9,HOAF_01,111,Pulse,23,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,390011.0,280535.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
test=test.drop(0) # drop rows with all zeros
test=test.drop(['question_number(num)', 'block_name(str)', 'sequence(num)', 'solution(str)', 'answer_options(str)', 'duration_question(num)', 'question_time(num)', 'response_time(num)', 'difficulty(num)', 'angle_diff(num)', 'dec_included(str)', 'Uncertainty', 'Duration', 'Uncertainty.1', 'ReqTime', 'ReqDur', 'Stim Type', 'Pair Index'], axis=1) # drop all defined columns 
#test=test.fillna(0) # fill non defined index with 0

## get time of first index

In [86]:
test['Time'][test['Event Type'][test['Event Type'] == 'Pulse'].index[0]]

375511.0

In [87]:
events = pd.DataFrame(columns=['onset', 'duration', 'trial_type', 'reaction_sub'], index=range(1,21)) # create DataFrame called "events", with specified columns

In [74]:
lf_df = pd.read_csv('/home/jan/Documents/Doktorarbeit/Daten/logfiles/HOAF_01-IPT_Kinect_2018_06_26_p2.log', delimiter='\t', skiprows=2)
lf_df
for index, rows in lf_df.iterrows():
    if rows["Code"] == 'Wait for PULSE_501':
        split_index=index


In [90]:
lf_df=lf_df.rename(index=str, columns={"Event Type": "event_type"})

In [91]:
lf_df[lf_df.event_type == 'Pulse'].shape[0]

501

In [77]:
def analyses_HOAF_log(log_file, sub, run_nr): # put everything together and define analyses_HOAF_log
    
    import os
    import csv
    import pandas as pd
    
    lf_df=pd.read_csv(log_file, delimiter='\t', skiprows=3) #define logfile
    
    lf_df=lf_df.drop(0)
    lf_df=lf_df.drop(['question_number(num)', 'block_name(str)', 'sequence(num)', 'solution(str)', 'answer_options(str)', 'duration_question(num)', 'question_time(num)', 'response_time(num)', 'difficulty(num)', 'angle_diff(num)', 'dec_included(str)', 'Uncertainty', 'Duration', 'Uncertainty.1', 'ReqTime', 'ReqDur', 'Stim Type', 'Pair Index'], axis=1)  
    lf_df=lf_df.fillna(0)
    
    events = pd.DataFrame(columns=['onset', 'duration', 'trial_type', 'condition', 'reaction_sub', 'reaction_gremium'], index=range(1,len(lf_df)))
                                                                                                            
    
    # cut the dataframe after the first run, which is indicated by 'Wartepause_2'
    for index, rows in lf_df.iterrows():
        if rows["Code"] == 'Wait for PULSE_501':
                split_index=index
    
    if run_nr == 1:   
        lf_df = lf_df.iloc[:split_index, :]


    ## get the time of the first scan to adjust event times
    time_first_scan = lf_df['Time'][lf_df['Event Type'][lf_df['Event Type'] == 'Pulse'].index[0]]
    
    count=1
    q_time = 0
    calulatetd_duration =0
    response_value=0
    duraton_written = 0
    
    for index, rows in lf_df.iterrows():
            #neu
            if 'Response' in str(rows["Event Type"]) and response_value == 1:
                #duration berechnen
                
                calulatetd_duration= (lf_df["Time"][index]/10000) - q_time
                q_time = 0 
                response_value = 0
                duration_written = 0
            if 'Manual' in str(rows["Event Type"]) and 'Pause' in str(rows["Code"]) and duration_written == 1:
                calulatetd_duration= (lf_df["Time"][index]/10000) - q_time
                q_time = 0 
                response_value = 0
                duration_written = 0
            if 'right_answer;' in str(rows["Code"]) and 'Math_Stress' in str(rows["Code"]) and 'timeout' not in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':
                events["trial_type"][count]='answer'
                events["reaction_sub"][count]='right_math'
                events["condition"][count]='non'
                events["reaction_gremium"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000)
                if count == 1:
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                q_time=(lf_df["Time"][index]/10000)
                count=count+1
                response_value = 1
                calulatetd_duration=0   
                duration_written = 1
            if 'right_answer;' in str(rows["Code"]) and 'Rotation_Stress' in str(rows["Code"]) and 'timeout' not in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':
                events["trial_type"][count]='answer'
                events["reaction_sub"][count]='right_rotation'
                events["condition"][count]='non'
                events["reaction_gremium"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000)
                if count == 1 :
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                q_time=(lf_df["Time"][index]/10000)
                count=count+1
                response_value = 1
                calulatetd_duration=0   
                duration_written=1
            if 'wrong_answer;' in str(rows["Code"]) and 'Rotation_Stress' in str(rows["Code"]) and 'timeout' not in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':
                events["trial_type"][count]='answer'
                events["reaction_sub"][count]='wrong_rotation'
                events["condition"][count]='non'
                events["reaction_gremium"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000)
                if count == 1 :
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                q_time=(lf_df["Time"][index]/10000)
                count=count+1
                response_value = 1
                calulatetd_duration=0  
                duration_written=1
            if 'question;' in str(rows["Code"]) and 'blocktimeout' not in str(rows["Code"]) and 'timeout' not in str(rows["Code"]) and 'Math_Control_' in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':
                events["trial_type"][count]='relax'
                events["condition"][count]='math'
                events["reaction_sub"][count]='non'
                events["reaction_gremium"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000)
                if count == 1 :
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                q_time=(lf_df["Time"][index]/10000)
                count=count+1
                response_value = 1
                calulatetd_duration=0
                duration_written=1
            if 'question;' in str(rows["Code"]) and 'blocktimeout' not in str(rows["Code"]) and 'timeout' not in str(rows["Code"]) and 'timeout' not in str(rows["Code"])  and 'Rotation_Stress_' in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':
                events["trial_type"][count]='stress'
                events["condition"][count]='rotation'
                events["reaction_sub"][count]='non'
                events["reaction_gremium"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000)
                if count == 1 :
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                q_time= (lf_df["Time"][index]/10000)
                count=count+1
                response_value = 1
                calulatetd_duration=0
                duration_written=1
            if 'vl_faster' in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':
                events["reaction_gremium"][count]='faster'
                events["reaction_sub"][count]='non'
                events["condition"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000)
                events["trial_type"][count]='stress_gremium'
                if count == 1 :
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                q_time=(lf_df["Time"][index]/10000)
                count=count+1
                response_value = 1 
                calulatetd_duration=0
                duration_written
            if 'vl_error' in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':
                events["reaction_gremium"][count]='wrong'
                events["reaction_sub"][count]='non'
                events["condition"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000)
                events["trial_type"][count]='stress_gremium'
                if count == 1 :
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                q_time=(lf_df["Time"][index]/10000)
                count=count+1
                response_value = 1 
                calulatetd_duration=0
                duration_written=1
            if 'wrong' in str(rows["Code"]) and 'Math_Stress_' in str(rows["Code"]) and 'timeout' not in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':#
                events["trial_type"][count]='answer'
                events["reaction_sub"][count]='wrong_math'
                events["condition"][count]='non'
                events["reaction_gremium"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000)
                if count == 1 :
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                q_time=(lf_df["Time"][index]/10000)
                count=count+1
                response_value = 1 
                calulatetd_duration=0
            if 'question;' in str(rows["Code"]) and 'blocktimeout' not in str(rows["Code"]) and 'timeout' not in str(rows["Code"]) and 'Math_Stress_' in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':
                events["trial_type"][count]='stress'
                events["condition"][count]='math'
                events["reaction_sub"][count]='non'
                events["reaction_gremium"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000)
                if count == 1 : 
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                q_time=(lf_df["Time"][index]/10000)
                count=count+1
                response_value = 1 
                calulatetd_duration=0
                duration_written=1
            if 'question;' in str(rows["Code"]) and 'blocktimeout' not in str(rows["Code"]) and 'timeout' not in str(rows["Code"]) and 'Rotation_Control_' in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':
                events["trial_type"][count]='relax'
                events["condition"][count]='rotation'
                events["reaction_sub"][count]='non'
                events["reaction_gremium"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000)
                if count == 1 :
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                q_time=(lf_df["Time"][index]/10000)
                count=count+1
                response_value = 1 
                calulatetd_duration=0
                duration_written=1
            if 'question;' in str(rows["Code"]) and 'timeout' in str(rows["Code"]) and 'blocktimeout' not in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':
                events["trial_type"][count]='timeout'
                events["condition"][count]='non'
                events["reaction_sub"][count]='non'
                events["reaction_gremium"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000)
                calulatetd_duration= (lf_df["Time"][index]/10000) - q_time
                q_time = (lf_df["Time"][index]/10000) 
                duration_written=0
                if count == 1 :
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                q_time=(lf_df["Time"][index]/10000)
                cal_dur=0
                count=count+1
                response_value = 1 
                calulatetd_duration=0
            if 'question;' in str(rows["Code"]) and 'blocktimeout' in str(rows["Code"]) and str(rows["Event Type"]) == 'Nothing':
                events["trial_type"][count]='blocktimeout'
                events["condition"][count]='non'
                events["reaction_sub"][count]='non'
                events["reaction_gremium"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000) 
                calulatetd_duration= (lf_df["Time"][index]/10000) - q_time
                q_time=(lf_df["Time"][index]/10000)
                duration_written=0
                if count == 1 :
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                count=count+1
                response_value = 1
                calulatetd_duration=0
            if 'Wartepause_' in str(rows["Code"]) and str(rows["Event Type"]) == 'Manual':
                events["trial_type"][count]='Wartepause'
                events["condition"][count]='non'
                events["reaction_sub"][count]='non'
                events["reaction_gremium"][count]='non'
                events["onset"][count]=((lf_df["Time"][index]-time_first_scan)/10000) 
                if count == 1 :
                    events["duration"][count]=calulatetd_duration
                else: 
                    events["duration"][count-1]=calulatetd_duration
                cal_dur=0
                count=count+1
                response_value = 1
                calulatetd_duration=0
                duration_written=1
    
    events=events.dropna(how='all') 
    events.to_csv(sub+'_task-stress_run-'+str(run_nr)+'_events.tsv',sep='\t', index = False)   
    return events

In [78]:
df = analyses_HOAF_log('/home/jan/Documents/Doktorarbeit/Daten/logfiles/HOAF_01-IPT_Kinect_2018_06_26_p2.log', 'sub-01', 2)

In [79]:
ls

analyses_HOAF_log_file_8.ipynb
analyses_HOAF_log_start_stop_stresstests_versuch1 (1).ipynb
analyses_HOAF_log_start_stop_stresstests_versuch3-Copy1.ipynb
analyses_HOAF_log_start_stop_stresstests_versuch3-Copy2.ipynb
analyses_HOAF_log_start_stop_stresstests_versuch3.ipynb
analyses_HOAF_log_start_stop_stresstests_versuch4-Copy3.ipynb
analyses_HOAF_log_start_stop_stresstests_versuch5-Copy1.ipynb
analyses_HOAF_log_start_stop_stresstests_versuch5.ipynb
analyses_HOAF_log_start_stop_stresstests_versuch6-Copy2.ipynb
analyses_HOAF_log_start_stop_stresstests_versuch7.ipynb
Änderung.ipynb
logfile änderung
logfile mit Differenzen.ods
output1.xlsx
output2.xlsx
sub-01_task-stress_run-1_events.tsv
sub-01_task-stress_run-2_events.tsv
sub-02_task-stress_run-2_events.tsv
Untitled-Copy1.ipynb
Untitled.ipynb


In [80]:
df


,onset,duration,trial_type,condition,reaction_sub,reaction_gremium
1,6.5325,2.3932,stress,rotation,non,non
2,8.9765,0.8523,answer,non,right_rotation,non
3,10.1065,0.2583,stress_gremium,non,non,faster
4,10.7808,2.1085,stress,rotation,non,non
5,12.9427,0.8617,answer,non,wrong_rotation,non
6,14.0721,0.474,stress_gremium,non,non,wrong
7,14.9719,0.3333,stress_gremium,non,non,faster
8,15.9694,4.1256,stress,rotation,non,non
9,20.1407,0.5964,answer,non,right_rotation,non
10,21.0371,0.0957,stress_gremium,non,non,faster


In [81]:
df.to_excel('output2.xlsx', engine='xlsxwriter') 

In [33]:
pwd

u'/home/jan/Documents/Doktorarbeit/Analyse/1level/logfile Analyse'